In this quickstart, we will
- run a hyperparameter sweep on the training of a model
- compare the results of the runs in the mlflow UI
- choose the best run and register it as a model
- deploy the model to a REST API
- build a container image suitable for deployment to the cloud platform

In [1]:
# importing the relevant libraries
import pandas as pd
import numpy as np

import keras
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [15]:
# loading the data
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4223,5.3,0.43,0.11,1.10,0.029,6.0,51.0,0.99076,3.51,0.48,11.20,4
4,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.99560,3.19,0.40,9.90,6
2951,7.2,0.20,0.22,1.60,0.044,17.0,101.0,0.99471,3.37,0.53,10.00,5
4368,6.5,0.30,0.28,11.45,0.041,29.0,109.0,0.99418,2.98,0.30,10.90,6
3585,7.0,0.24,0.35,1.00,0.032,42.0,104.0,0.98988,3.16,0.37,11.70,7
4437,6.5,0.29,0.30,9.15,0.051,25.0,166.0,0.99339,3.24,0.56,11.35,6
1361,6.0,0.13,0.28,5.70,0.038,56.0,189.5,0.99480,3.59,0.43,10.60,7
2685,7.0,0.16,0.36,2.60,0.029,28.0,98.0,0.99126,3.11,0.37,11.20,7
4124,6.0,0.13,0.36,1.60,0.052,23.0,72.0,0.98974,3.10,0.50,11.50,7
4019,6.2,0.32,0.12,4.80,0.054,6.0,97.0,0.99424,3.16,0.50,9.30,5


In [16]:
# splitting the dataset
train, test = train_test_split(data, test_size=0.25, random_state=42)

X_train = train.drop(['quality'], axis=1).values
X_test = test.drop(columns=['quality'], axis=1).values

y_train = train[['quality']].values.ravel()
y_test = test[['quality']].values.ravel()

# validation data
train_x, val_x, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# creating signature
signature = infer_signature(train_x, train_y)

In [27]:
# building the ANN model

import mlflow.tensorflow


def train_model(params, epochs, train_x, train_y, val_x, val_y, test_x, test_y):
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],
        momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    # train the model with learning rate and momentus
    with mlflow.start_run(run_name="ANN Wine Quality Prediction Model", nested=True):
        model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=epochs, batch_size=64)

        # evaluating the model
        eval_results = model.evaluate(val_x, val_y, batch_size=64)

        eval_rmse = eval_results[1]

        # logging the model and params
        mlflow.log_params(params)
        mlflow.log_metric("Evaluation RMSE", eval_rmse)

        mlflow.tensorflow.log_model(model=model, artifact_path="model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}
        

In [28]:
# creating the objective function

def objective(params):
    result = train_model(
        params=params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        val_x=val_x,
        val_y=val_y,
        test_x=X_test,
        test_y=y_test
    )

    return result

In [29]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0)
}

In [30]:
import mlflow.tensorflow


mlflow.set_experiment("wine-quality")

with mlflow.start_run(run_name="Wine Quality Prediction"):
    trials = Trials()

    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # logging the best parameter
    mlflow.log_params(best)
    mlflow.log_metric("Evaluation RMSE", best_run['loss'])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature)

    # printing the results
    print(f"Best Parameters: {best}")
    print(f"Best Evaluation RMSE: {best_run['loss']}")

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 43s 968ms/step - loss: 38.8103 - root_mean_squared_error: 6.2298
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 39.5440 - root_mean_squared_error: 6.2881
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 39.7161 - root_mean_squared_error: 6.3019
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 39.6715 - root_mean_squared_error: 6.2984   
 9/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 39.2315 - root_mean_squared_error: 6.2513 
10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 39.1549 - root_mean_squared_error: 6.2572 
 8/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 39.3174 - root_mean_squared_error: 6.2572 
12/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 39.0083 - root_mean_squared_error: 6.2454 
11/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 39.0810 - root_mean_squared_error: 6.2454 
13/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 38.9430 - root_mean_squared_error: 6.2402 
14/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - l

2025/06/22 18:41:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


2025/06/22 18:42:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\STUDSENT\AppData\Local\Temp\tmpkfukhaa8\model, flavor: tensorflow). Fall back to return ['tensorflow==2.18.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2025/06/22 18:42:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best Parameters: {'lr': 0.009518981481993188, 'momentum': 0.3068230200514316}
Best Evaluation RMSE: 0.9309758543968201
